In [1]:
# ATSC 500 Project Code 
# Selecting and reading CMIP6 data
# Mina Deshler

import xarray as xr
xr.set_options(display_style='html')
import intake
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import cartopy.crs as ccrs
import cartopy

In [3]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
...,...,...,...,...,...,...,...,...,...,...,...
514813,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,tas,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20211207
514814,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,tauu,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20211207
514815,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,hur,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20211207
514816,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,hus,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20211207


### Reference material for getting CMIP6 data:
CMIP6 all variable name info: https://wcrp-cmip.org/cmip-data-access/ 

TABLE & VARIABLE IDs: https://airtable.com/appYNLuWqAgzLbhSq/shrKcLEdssxb8Yvcp/tblL7dJkC3vl5zQLb

Grid labels: https://github.com/WCRP-CMIP/CMIP6_CVs/blob/main/CMIP6_grid_label.json 

Variables of interest for this project:
- surface upward sensible heat (3-hourly) - 3hr, hfss
- daily max 2m temp - day, tasmax
- daily min 2m temp - day, tasmin

In [4]:
cmipdf = col.df
cmipdf['table_id'].unique()

array(['Amon', '6hrPlev', '3hr', 'day', 'EmonZ', 'E3hr', '6hrPlevPt',
       'AERmon', 'LImon', 'CFmon', 'Lmon', 'fx', 'SImon', 'Ofx', 'Omon',
       'EdayZ', 'Emon', 'CFday', 'AERday', 'Eday', 'Oyr', 'Eyr', 'Oday',
       'SIday', 'AERmonZ', '6hrLev', 'E1hrClimMon', 'CF3hr', 'AERhr',
       'Odec', 'Oclim', 'Efx', 'Aclim', 'SIclim', 'IfxGre', 'ImonGre',
       'Eclim'], dtype=object)

In [5]:
# identify source id + experiment id for all options where variable id = tasmax and variable id = hfss
var1 = 'hfss'
var2 = '3hr'
findsh = cmipdf[(cmipdf['variable_id'] == var1) & (cmipdf['table_id'] == var2)]
print(findsh)
# can use the amip simulations from GFDL-CM4 (idx 17258, 17269)

      activity_id institution_id     source_id       experiment_id member_id  \
4014         CMIP      NOAA-GFDL      GFDL-CM4        abrupt-4xCO2  r1i1p1f1   
4015         CMIP      NOAA-GFDL      GFDL-CM4        abrupt-4xCO2  r1i1p1f1   
9834         CMIP      NOAA-GFDL      GFDL-CM4           piControl  r1i1p1f1   
10006        CMIP      NOAA-GFDL      GFDL-CM4          historical  r1i1p1f1   
10044        CMIP      NOAA-GFDL      GFDL-CM4          historical  r1i1p1f1   
10288        CMIP      NOAA-GFDL      GFDL-CM4           piControl  r1i1p1f1   
12347  HighResMIP      NOAA-GFDL  GFDL-CM4C192  highresSST-present  r1i1p1f1   
12709  HighResMIP      NOAA-GFDL  GFDL-CM4C192   highresSST-future  r1i1p1f1   
16489        CMIP      NOAA-GFDL      GFDL-CM4             1pctCO2  r1i1p1f1   
16490        CMIP      NOAA-GFDL      GFDL-CM4             1pctCO2  r1i1p1f1   
17268        CMIP      NOAA-GFDL      GFDL-CM4                amip  r1i1p1f1   
17269        CMIP      NOAA-GFDL      GF

In [6]:
#corresponding tasmax / tasmin 
var1 = 'tasmin'
var2 = 'day'
#var3 = 'GFDL-CM4'
var3 = 'historical'
findsh = cmipdf[(cmipdf['variable_id'] == var1) & (cmipdf['table_id'] == var2) & (cmipdf['experiment_id'] == var3)]
print(findsh)

       activity_id institution_id     source_id experiment_id  member_id  \
985           CMIP      NOAA-GFDL     GFDL-ESM4    historical   r2i1p1f1   
1770          CMIP      NOAA-GFDL     GFDL-ESM4    historical   r3i1p1f1   
9528          CMIP      NOAA-GFDL      GFDL-CM4    historical   r1i1p1f1   
9529          CMIP      NOAA-GFDL      GFDL-CM4    historical   r1i1p1f1   
22212         CMIP           IPSL  IPSL-CM6A-LR    historical   r8i1p1f1   
...            ...            ...           ...           ...        ...   
513898        CMIP            MRI    MRI-ESM2-0    historical   r7i1p1f1   
513915        CMIP            MRI    MRI-ESM2-0    historical   r8i1p1f1   
513932        CMIP            MRI    MRI-ESM2-0    historical   r9i1p1f1   
513945        CMIP            MRI    MRI-ESM2-0    historical  r10i1p1f1   
514458        CMIP            MRI    MRI-ESM2-0    historical   r6i1p1f1   

       table_id variable_id grid_label  \
985         day      tasmin        gr1   
177

In [7]:
# Getting data for 3-hourly sensible heat flux over BC
write = True
dset = []
# Extract only the summer months for each year
timeslices = [slice('2004-06-01', '2004-08-31'), slice('2005-06-01', '2005-08-31'), slice('2006-06-01', '2006-08-31'), slice('2007-06-01', '2007-08-31'),
              slice('2008-06-01', '2008-08-31'), slice('2009-06-01', '2009-08-31'), slice('2010-06-01', '2010-08-31'), slice('2011-06-01', '2011-08-31'),
              slice('2012-06-01', '2012-08-31'), slice('2013-06-01', '2013-08-31'), slice('2014-06-01', '2014-08-31')]
if write:
    # table_id - atmosphere daily, variable id - max daily temp, source id - model used, experiment id - 
    can_subset = col.search(table_id="3hr", variable_id = "hfss", source_id = "GFDL-CM4", experiment_id = 'amip')
    dset_dict = can_subset.to_dataset_dict(zarr_kwargs={'consolidated':True})
    #can_dset = dset_dict['CMIP.CCCma.CanESM5.historical.Amon.gn']
    can_dset = dset_dict['CMIP.NOAA-GFDL.GFDL-CM4.amip.3hr.gr1']

    # Setting coords for BC + years of interest
    #can_bc_dset = can_dset.sel(lon = slice(225.,244.6875), lat = slice(48.8, 60), time = slice('2004-06-01', '2014-08-31'))
    for i in timeslices:
        #Lon: -139 to -114 = 221 to 246 
        can_bc_dset = can_dset.sel(lon = slice(221,246), lat = slice(48.8, 60), time = i)
        dset.append(can_bc_dset)
    datasave = xr.concat(dset,dim="time")
    print("got here")
    datasave.load().to_zarr("amip_hfss_best.zarr")


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


got here


In [8]:
#check
hfss = xr.open_zarr('amip_hfss_best.zarr')
hfss

<xarray.Dataset>
Dimensions:         (bnds: 2, dcpp_init_year: 1, member_id: 1, time: 8096,
                     lat: 11, lon: 20)
Coordinates:
  * bnds            (bnds) float64 1.0 2.0
  * dcpp_init_year  (dcpp_init_year) float64 nan
  * lat             (lat) float64 49.5 50.5 51.5 52.5 ... 56.5 57.5 58.5 59.5
    lat_bnds        (lat, bnds) float64 dask.array<chunksize=(11, 2), meta=np.ndarray>
  * lon             (lon) float64 221.9 223.1 224.4 225.6 ... 243.1 244.4 245.6
    lon_bnds        (lon, bnds) float64 dask.array<chunksize=(20, 2), meta=np.ndarray>
  * member_id       (member_id) object 'r1i1p1f1'
  * time            (time) object 2004-06-01 01:30:00 ... 2014-08-31 22:30:00
    time_bnds       (time, bnds) object dask.array<chunksize=(8096, 1), meta=np.ndarray>
Data variables:
    hfss            (member_id, dcpp_init_year, time, lat, lon) float32 dask.array<chunksize=(1, 1, 2024, 6, 10), meta=np.ndarray>
Attributes: (12/60)
    Conventions:                      CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:                      CMIP
    branch_method:                    no parent
    branch_time_in_child:             0.0
    comment:                          <null ref>
    contact:                          gfdl.climate.model.info@noaa.gov
    ...                               ...
    title:                            NOAA GFDL GFDL-CM4 model output prepare...
    tracking_id:                      hdl:21.14100/a97aa028-bb0e-4dd9-9ffb-61...
    variable_id:                      hfss
    variant_info:                     N/A
    variant_label:                    r1i1p1f1
    version_id:                       v20180701

In [9]:
# Getting data for daily Tmax & Tmin over BC
write = True
tas_dset = []
variables = ["tasmax","tasmin"]
# Extract only the summer months for each year
timeslices = [slice('2004-06-01', '2004-08-31'), slice('2005-06-01', '2005-08-31'), slice('2006-06-01', '2006-08-31'), slice('2007-06-01', '2007-08-31'),
              slice('2008-06-01', '2008-08-31'), slice('2009-06-01', '2009-08-31'), slice('2010-06-01', '2010-08-31'), slice('2011-06-01', '2011-08-31'),
              slice('2012-06-01', '2012-08-31'), slice('2013-06-01', '2013-08-31'), slice('2014-06-01', '2014-08-31')]
if write:
    # table_id - atmosphere daily, variable id - max daily temp, source id - model used, experiment id - 
    tcan_subset = col.search(table_id="day", variable_id = variables, source_id = "GFDL-CM4", experiment_id = 'historical')
    tdset_dict = tcan_subset.to_dataset_dict(zarr_kwargs={'consolidated':True})
    tcan_dset = tdset_dict['CMIP.NOAA-GFDL.GFDL-CM4.historical.day.gr1']

    # Setting coords for BC + years of interest
    #tas_bc_dset = tcan_dset.sel(lon = slice(225.,244.6875), lat = slice(48.835241, 59.99702), time = timeslices)
    for i in timeslices:
        tas_bc_dset = tcan_dset.sel(lon = slice(221,246), lat = slice(48.8, 60), time = i)
        tas_dset.append(tas_bc_dset)
    datasave = xr.concat(tas_dset,dim="time")
    print("got here")
    datasave.load().to_zarr("maxmintemps2.zarr")


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


got here


In [10]:
#check
cmiptemps = xr.open_zarr('maxmintemps2.zarr')
cmiptemps

<xarray.Dataset>
Dimensions:         (dcpp_init_year: 1, lat: 11, bnds: 2, lon: 20,
                     member_id: 1, time: 1012)
Coordinates:
  * dcpp_init_year  (dcpp_init_year) float64 nan
  * lat             (lat) float64 49.5 50.5 51.5 52.5 ... 56.5 57.5 58.5 59.5
    lat_bnds        (lat, bnds) float64 dask.array<chunksize=(11, 2), meta=np.ndarray>
  * lon             (lon) float64 221.9 223.1 224.4 225.6 ... 243.1 244.4 245.6
    lon_bnds        (lon, bnds) float64 dask.array<chunksize=(20, 2), meta=np.ndarray>
  * member_id       (member_id) object 'r1i1p1f1'
  * time            (time) object 2004-06-01 12:00:00 ... 2014-08-31 12:00:00
    time_bnds       (time, bnds) object dask.array<chunksize=(1012, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    tasmax          (member_id, dcpp_init_year, time, lat, lon) float32 dask.array<chunksize=(1, 1, 506, 6, 20), meta=np.ndarray>
    tasmin          (member_id, dcpp_init_year, time, lat, lon) float32 dask.array<chunksize=(1, 1, 506, 6, 20), meta=np.ndarray>
Attributes: (12/55)
    Conventions:                      CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            36500.0
    comment:                          <null ref>
    ...                               ...
    sub_experiment_id:                none
    table_id:                         day
    title:                            NOAA GFDL GFDL-CM4 model output prepare...
    variant_info:                     N/A
    variant_label:                    r1i1p1f1
    version_id:                       v20180701